In [129]:
import pandas as pd

train = pd.read_csv('datas/train.csv')
test = pd.read_csv('datas/test.csv')
submission = pd.read_csv('datas/sample_submission.csv')

display(train.head())
display(test.head())
display(submission.head())

,id,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,1,53,1,2,130,197,1,0,152,0,1.2,0,0,2,1
1,2,52,1,3,152,298,1,1,178,0,1.2,1,0,3,1
2,3,54,1,1,192,283,0,0,195,0,0.0,2,1,3,0
3,4,45,0,0,138,236,0,0,152,1,0.2,1,0,2,1
4,5,35,1,1,122,192,0,1,174,0,0.0,2,0,2,1


,id,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal
0,1,57,1,0,150,276,0,0,112,1,0.6,1,1,1
1,2,59,1,3,170,288,0,0,159,0,0.2,1,0,3
2,3,57,1,2,150,126,1,1,173,0,0.2,2,1,3
3,4,56,0,0,134,409,0,0,150,1,1.9,1,2,3
4,5,71,0,2,110,265,1,0,130,0,0.0,2,1,2


,id,target
0,1,0
1,2,0
2,3,0
3,4,0
4,5,0


In [130]:
# 결측치 확인
display(train.isnull().sum())
display(test.isnull().sum())

id          0
age         0
sex         0
cp          0
trestbps    0
chol        0
fbs         0
restecg     0
thalach     0
exang       0
oldpeak     0
slope       0
ca          0
thal        0
target      0
dtype: int64

id          0
age         0
sex         0
cp          0
trestbps    0
chol        0
fbs         0
restecg     0
thalach     0
exang       0
oldpeak     0
slope       0
ca          0
thal        0
dtype: int64

표준화 -> 범주형 변수는 그대로 두고 연속형 변수만 표준화
(age, trestbps, chol, thalach, oldpeak)

In [131]:
from sklearn.preprocessing import StandardScaler

scale_data = ['age', 'trestbps', 'chol', 'thalach', 'oldpeak']

scaler = StandardScaler()
train[scale_data] = scaler.fit_transform(train[scale_data])
test[scale_data] = scaler.transform(test[scale_data])

특성과 타겟 분리 후 학습 데이터, 검증 데이터로 분할

In [132]:
from sklearn.model_selection import train_test_split

X = train.drop(['id', 'target'], axis=1)
y = train['target']

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

로지스틱 회귀 분석

In [133]:
from sklearn.linear_model import LogisticRegression

LR_model = LogisticRegression()
LR_model.fit(X_train, y_train)

LR_predict = LR_model.predict(X_valid)
LR_predict

array([1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0,
       1, 1, 1, 0, 0, 0, 1, 1, 0], dtype=int64)

결정 트리 분석

In [98]:
from sklearn.tree import DecisionTreeClassifier

DT_model = DecisionTreeClassifier(random_state=42)
DT_model.fit(X_train, y_train)

DT_predict = DT_model.predict(X_valid)
DT_predict

array([1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 0], dtype=int64)

랜덤 포레스트 분석

In [99]:
from sklearn.ensemble import RandomForestClassifier

RF_model = RandomForestClassifier(random_state=10)
RF_model.fit(X_train, y_train)

RF_predict = RF_model.predict(X_valid)
RF_predict

array([1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0,
       1, 1, 1, 0, 0, 1, 1, 1, 0], dtype=int64)

로지스틱 회귀 성능 평가

In [134]:
from sklearn.metrics import f1_score

f1_score_LR = f1_score(y_valid, LR_predict)
display(f'fl_score: {f1_score_LR}')

'fl_score: 0.875'

결정 트리 성능 평가

In [89]:
from sklearn.metrics import f1_score

f1_score_DT = f1_score(y_valid, DT_predict)
display(f'fl_score: {f1_score_DT}')

'fl_score: 0.7647058823529411'

랜덤 포레스트 성능 평가

In [90]:
from sklearn.metrics import f1_score

f1_score_RF = f1_score(y_valid, RF_predict)
display(f'fl_score: {f1_score_RF}')

'fl_score: 0.7567567567567568'

In [135]:
test = test.drop('id', axis=1)
predict = LR_model.predict(test)

submission['target'] = predict
submission.to_csv('submission.csv', index=False, quoting=2, encoding="utf-8-sig", na_rep=0, float_format='%.6f')